In [1]:
import numpy as np

In [2]:
class MultiClass:
    
    def __init__(self, rate=0.01, cycles=1000):
        self.rate = rate
        self.cycles = cycles
    
    def run(self, points, ids):
        self.theta = np.zeros((4, len(points[0])))
        self.best_theta = np.zeros((4, len(points[0])))
        self.bias = np.zeros(4)
        self.best_bias = np.zeros(4)
        self.best_miss_count = len(ids)
        
        self.loss = np.zeros(self.cycles)
        
        for i in range(self.cycles):
            for point, truth in zip(points, ids):
                self.compute_loss(i, point, truth)
                error = truth - self.predict(point)
                if error == 0:
                    shift = 0
                else:
                    shift = self.rate
                
                for index in [0, 1, 2, 3]:
                    if index != truth:
                        dot = np.dot(point, self.theta[index]) + self.bias[index]
                        self.bias[index] -= shift * np.exp(dot)
                        self.theta[index] -= shift * point * np.exp(dot)
                
                self.compute_miss(points, ids)

        self.report(points, ids)
        return self
    
    def compute_loss(self, i, x, y):
        dot0 = np.dot(x, self.theta[0]) + self.bias[0]
        dot1 = np.dot(x, self.theta[1]) + self.bias[1]
        dot2 = np.dot(x, self.theta[2]) + self.bias[2]
        dot3 = np.dot(x, self.theta[3]) + self.bias[3]
        exp_sum = np.exp(dot0) + np.exp(dot1) + np.exp(dot2) + np.exp(dot3)
        self.loss[i] += exp_sum - np.exp(np.dot(x, self.theta[y]) + self.bias[y])
    
    def predict(self, ds):
        dot0 = np.dot(ds, self.theta[0]) + self.bias[0]
        dot1 = np.dot(ds, self.theta[1]) + self.bias[1]
        dot2 = np.dot(ds, self.theta[2]) + self.bias[2]
        dot3 = np.dot(ds, self.theta[3]) + self.bias[3]
        dot_list = [dot0, dot1, dot2, dot3]
        return np.argmax(dot_list, axis=0)
    
    def compute_miss(self, points, ids):
        miss_count = 0
        miss = []
        for point, truth in zip(points, ids):
            prediction = self.predict(point)
            if truth - prediction != 0:
                miss_count += 1
                miss.append(point)
        if miss_count <= self.best_miss_count:
            self.best_miss_count = miss_count
            self.best_miss = miss
            self.best_bias[0] = self.bias[0]
            self.best_theta[0] = self.theta[0]
            self.best_bias[1] = self.bias[1]
            self.best_theta[1] = self.theta[1]
            self.best_bias[2] = self.bias[2]
            self.best_theta[2] = self.theta[2]
            self.best_bias[3] = self.bias[3]
            self.best_theta[3] = self.theta[3]
    
    def report(self, points, ids):
        miss = []
        for point, truth in zip(points, ids):
            prediction = self.predict(point)
            if truth - prediction != 0:
                miss.append([point, prediction, truth])
        if len(miss) > 0:
            print("Mislabeled:")
            for c in miss:
                print("{0}, prediction: {1}, truth: {2}".format(c[0], c[1], c[2]))

In [3]:
multi = MultiClass()

In [4]:
data = np.load('data.npz')
trainx = data['x']
trainy = data['y']
testx = data['testx']
testy = data['testy']

In [ ]:
out = multi.run(trainx, trainy)

In [ ]:
print(out.theta)
print(out.bias)
print(out.best_theta)
print(out.best_bias)
print(out.best_miss)

print(len(out.best_miss))
print(len(trainy))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions

In [ ]:
plt.plot(out.loss)
plt.xscale('log')
plt.xlabel('Cycles')
plt.ylabel('Loss')

In [ ]:
plot_decision_regions(testx, testy, clf=multi)